In [101]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from imblearn.ensemble import BalancedRandomForestClassifier
pd.set_option('display.max_columns', 500)

In [4]:
DATA_DIR = os.path.join('..', 'data')

In [5]:
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))

D:\presenso\progs\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,1,2,4,11,16,17,20,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,58,59,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [162]:
test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

D:\presenso\progs\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,1,2,4,11,16,17,20,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,58,59,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [163]:
train.target.value_counts()

0    5724146
1      78254
Name: target, dtype: int64

In [164]:
len(train[train['target'] == 1]) / len(train)

0.013486488349648421

In [165]:
# Totally inbalanced dataset. Will firstly verify distance hypothesis on data with target and then prepare a model

In [166]:
train.head(1)

,location_number,latitude_x,longitude_x,id,latitude_y,longitude_y,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,commission,discount_percentage,status,verified,rank,vendor_rating,country_id,city_id,target
0,0,-96.44,-67.2,4,-0.5884,0.7544,2.0,0.0,6.0,1.0,15,0.0,0.0,1.0,1,11,4.4,1.0,1.0,0


In [167]:
test = test.dropna()
test_ids = test['CID X LOC_NUM X VENDOR']
cols_to_leave = ['location_number', 'latitude_x', 'longitude_x', 'id', 'latitude_y', 'longitude_y', 'vendor_category_id', 'delivery_charge', 'serving_distance', 'is_open', 'prepration_time', 'commission', 'discount_percentage', 'status', 'verified', 'rank', 'vendor_rating', 'country_id', 'city_id']

In [168]:
len(test)

168264

In [169]:
len(test_ids)

168264

In [170]:
train = train[cols_to_leave + ['target']]

In [171]:
test = test[cols_to_leave]

In [172]:
train.head(1)

,location_number,latitude_x,longitude_x,id,latitude_y,longitude_y,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,commission,discount_percentage,status,verified,rank,vendor_rating,country_id,city_id,target
0,0,-96.44,-67.2,4,-0.5884,0.7544,2.0,0.0,6.0,1.0,15,0.0,0.0,1.0,1,11,4.4,1.0,1.0,0


In [173]:
train.commission = train.commission.fillna(0)
test.commission = test.commission.fillna(0)

In [174]:
train.latitude_x = train.latitude_x.fillna(0)
train.latitude_y = train.latitude_y.fillna(0)
train.longitude_x = train.longitude_x.fillna(0)
train.longitude_y = train.longitude_y.fillna(0)

test.latitude_x = test.latitude_x.fillna(0)
test.latitude_y = test.latitude_y.fillna(0)
test.longitude_x = test.longitude_x.fillna(0)
test.longitude_y = test.longitude_y.fillna(0)

In [175]:
train['deliv_dist_estimation_euc'] = np.sqrt(np.power(train['latitude_x'] - train['latitude_y'], 2) + 
                                      np.power(train['longitude_x'] - train['longitude_y'], 2))

train['deliv_dist_estimation_mht_sqr'] = np.sqrt(np.abs(train['latitude_x'] - train['latitude_y']) + 
                                      np.abs(train['longitude_x'] - train['longitude_y']))

train['deliv_dist_estimation_mht'] = np.abs(train['latitude_x'] - train['latitude_y']) + \
                                      np.abs(train['longitude_x'] - train['longitude_y'])
    
train['deliv_dist_estimation_geo'] = 2 * np.arcsin(np.sqrt(np.power(np.sin((train.latitude_x - train.longitude_x) / 2), 2) + 
                                                    np.power(np.sin((train.latitude_y - train.longitude_y) / 2), 2) * 
                                                    np.cos(train.latitude_x) * 
                                                    np.cos(train.longitude_x)))



test['deliv_dist_estimation_euc'] = np.sqrt(np.power(test['latitude_x'] - test['latitude_y'], 2) + 
                                      np.power(test['longitude_x'] - test['longitude_y'], 2))

test['deliv_dist_estimation_mht_sqr'] = np.sqrt(np.abs(test['latitude_x'] - test['latitude_y']) + 
                                      np.abs(test['longitude_x'] - test['longitude_y']))

test['deliv_dist_estimation_mht'] = np.abs(test['latitude_x'] - test['latitude_y']) + \
                                      np.abs(test['longitude_x'] - test['longitude_y'])
    
test['deliv_dist_estimation_geo'] = 2 * np.arcsin(np.sqrt(np.power(np.sin((test.latitude_x - test.longitude_x) / 2), 2) + 
                                                    np.power(np.sin((test.latitude_y - test.longitude_y) / 2), 2) * 
                                                    np.cos(test.latitude_x) * 
                                                    np.cos(test.longitude_x)))

In [142]:
train.dtypes

location_number                    int64
latitude_x                       float64
longitude_x                      float64
id                                 int64
latitude_y                       float64
longitude_y                      float64
vendor_category_id               float64
delivery_charge                  float64
serving_distance                 float64
is_open                          float64
prepration_time                    int64
commission                       float64
discount_percentage              float64
status                           float64
verified                           int64
rank                               int64
vendor_rating                    float64
country_id                       float64
city_id                          float64
target                             int64
deliv_dist_estimation_euc        float64
deliv_dist_estimation_mht_sqr    float64
deliv_dist_estimation_mht        float64
deliv_dist_estimation_geo        float64
dtype: object

In [143]:
train['latitude_x'].isna().sum()  # this will result in big problems for us!

0

In [144]:
test['city_id'].isna().sum()  # this will result in big problems for us!

40300

In [95]:
X = train
y = X.target
X = X.drop(columns=['target'])

In [96]:
X.shape

(5802400, 23)

In [97]:
y.shape

(5802400,)

In [98]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
kf.get_n_splits(X)

5

In [99]:
def model_fit(x_train, y_train, classifier, **params):
    clf2 = classifier(**params)
    clf2 = clf2.fit(x_train, y_train)
    return clf2


def model_predict(data, model):
    return model.predict(data)

In [102]:
clf = BalancedRandomForestClassifier
clf_params = {
    "n_estimators": 200,
    "n_jobs": 8
}

fold = 0
scores = []

for train_index, test_index in kf.split(X):
    fold += 1
    X_train, X_val = X.loc[X.index.intersection(train_index)], X.loc[X.index.intersection(test_index)]
    y_train, y_val = y.loc[y.index.intersection(train_index)], y.loc[y.index.intersection(test_index)]

    print(f'FOLD #{fold}')
    print("train:", len(X_train))
    print("val:", len(X_val))
    model = model_fit(X_train, y_train, classifier=clf, **clf_params)

    y_pred = model_predict(X_val, model)
    score = f1_score(y_val, y_pred)
    print('Validation Score: {}'.format(score))
    scores.append(score)

FOLD #1
train: 4641920
val: 1160480
Validation Score: 0.0939191525638436
FOLD #2
train: 4641920
val: 1160480
Validation Score: 0.09365433260378954
FOLD #3
train: 4641920
val: 1160480
Validation Score: 0.09428609378884738
FOLD #4
train: 4641920
val: 1160480
Validation Score: 0.0930564815696733
FOLD #5
train: 4641920
val: 1160480
Validation Score: 0.09286337881096257


In [105]:
print(f'MEAN OF SCOREs: {np.mean(scores)}')

model = model_fit(X, y, classifier=clf, **clf_params)

MEAN OF SCOREs: 0.09355588786742328


In [176]:
preds = model.predict(test)

In [177]:
len(preds)

168264

In [178]:
len(test_ids)

168264

In [179]:
np.unique(preds, return_counts=True)

(array([0, 1], dtype=int64), array([110749,  57515], dtype=int64))

In [180]:
result = pd.DataFrame({'CID X LOC_NUM X VENDOR': test_ids, 'target': preds})
#result.to_csv(os.path.join(DATA_DIR, 'submission.csv'), index=False)

In [189]:
sbmsn = pd.read_csv(os.path.join(DATA_DIR, 'SampleSubmission.csv'))

In [190]:
sbmsn.head(1)

,CID X LOC_NUM X VENDOR,target
0,Z59FTQD X 0 X 243,0


In [191]:
sbmsn = pd.merge(sbmsn, result, on='CID X LOC_NUM X VENDOR', how='left')

In [192]:
sbmsn.head()

,CID X LOC_NUM X VENDOR,target_x,target_y
0,Z59FTQD X 0 X 243,0,NaN
1,0JP29SK X 0 X 243,0,NaN
2,0JP29SK X 1 X 243,0,NaN
3,0JP29SK X 2 X 243,0,NaN
4,0JP29SK X 3 X 243,0,NaN


In [201]:
sbmsn = sbmsn.fillna(0)
sbmsn['target_y'] = pd.to_numeric(sbmsn.target_y, downcast='integer')
sbmsn.head()

,CID X LOC_NUM X VENDOR,target_x,target_y
0,Z59FTQD X 0 X 243,0,0
1,0JP29SK X 0 X 243,0,0
2,0JP29SK X 1 X 243,0,0
3,0JP29SK X 2 X 243,0,0
4,0JP29SK X 3 X 243,0,0


In [197]:
sbmsn['target_y'].value_counts()

0.0    1614485
1.0      57515
Name: target_y, dtype: int64

In [202]:
sbmsn['target'] = sbmsn['target_y']

In [203]:
del sbmsn['target_x']
del sbmsn['target_y']

In [205]:
sbmsn.to_csv(os.path.join(DATA_DIR, 'submission_fin.csv'), index=False)